In [1]:
#!/usr/bin/env python
import json, openai,requests
import warnings
import os
warnings.filterwarnings('ignore')
import datetime
from time import sleep

In [19]:
openai_org = "org-num"
openai_key = "sk-num"

In [18]:
import tiktoken
import pandas as pd
def num_tokens_from_string(string: str, 
                           encoding_name: str = 'text-davinci-003') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [17]:
num_tokens_from_string("This is my super prompt from Insight Builder")

8

In [16]:
get_role = input("What role you want me to play: ")

What role you want me to play: Experienced Database Administrator and Programmer


In [4]:
#reading in the dataset after unpacking it from zip file

import shutil
import pandas as pd

shutil.unpack_archive("archive.zip")

In [5]:
with open('AWS_Honeypot_marx-geo.csv','r') as sd:
    data = sd.readlines()

In [6]:
data[0:5]

['datetime,host,src,proto,type,spt,dpt,srcstr,cc,country,locale,localeabbr,postalcode,latitude,longitude,\n',
 '3/3/13 21:53,groucho-oregon,1032051418,TCP,,6000,1433,61.131.218.218,CN,China,Jiangxi Sheng,36,,28.55,115.9333,\n',
 '3/3/13 21:57,groucho-oregon,1347834426,UDP,,5270,5060,80.86.82.58,DE,Germany,,,,51,9,\n',
 '3/3/13 21:58,groucho-oregon,2947856490,TCP,,2489,1080,175.180.184.106,TW,Taiwan,Taipei,,,25.0392,121.525,\n',
 '3/3/13 21:58,groucho-us-east,841842716,UDP,,43235,1900,50.45.128.28,US,United States,Oregon,OR,97124,45.5848,-122.9117,\n']

In [7]:
data[6:11]

['3/3/13 21:58,groucho-tokyo,3323217250,TCP,,32628,2323,198.20.69.98,US,United States,Illinois,IL,60661,41.8825,-87.6441,\n',
 '3/3/13 21:59,groucho-oregon,3730416887,TCP,,6000,1433,222.89.164.247,CN,China,Henan Sheng,41,,34.6836,113.5325,\n',
 '3/3/13 22:07,groucho-singapore,3738622573,TCP,,6000,3306,222.214.218.109,CN,China,Sichuan Sheng,51,,30.6667,104.0667,\n',
 '3/3/13 22:12,groucho-oregon,3683919430,TCP,,6000,1433,219.148.38.70,CN,China,Hebei,13,,39.8897,115.275,\n',
 '3/3/13 22:14,groucho-singapore,1007884304,TCP,,6000,1433,60.19.24.16,CN,China,Liaoning,21,,41.7922,123.4328,\n']

In [8]:
##Version the prompt listing questions

In [8]:
#splitting the dataset for managing the tokens

rng1 = 1
rng2 = rng1 + 55
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)



before spliting data 1 and 56


In [20]:
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and help me do the following
1. Create a Database table for this dataset in Postgres Database
2. I have data in csv format. Provide the commands to load the data 
3. Datamodel the table to 3 normal form and provide the schema
Answer:
"""

In [10]:
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')

Number of tokens for the dataset is 3471


In [21]:
print(prompt_made)


dataset=datetime,host,src,proto,type,spt,dpt,srcstr,cc,country,locale,localeabbr,postalcode,latitude,longitude,
+['3/3/13 21:53,groucho-oregon,1032051418,TCP,,6000,1433,61.131.218.218,CN,China,Jiangxi Sheng,36,,28.55,115.9333,\n', '3/3/13 21:57,groucho-oregon,1347834426,UDP,,5270,5060,80.86.82.58,DE,Germany,,,,51,9,\n', '3/3/13 21:58,groucho-oregon,2947856490,TCP,,2489,1080,175.180.184.106,TW,Taiwan,Taipei,,,25.0392,121.525,\n', '3/3/13 21:58,groucho-us-east,841842716,UDP,,43235,1900,50.45.128.28,US,United States,Oregon,OR,97124,45.5848,-122.9117,\n', '3/3/13 21:58,groucho-singapore,3587648279,TCP,,56577,80,213.215.43.23,FR,France,,,,48.86,2.35,\n', '3/3/13 21:58,groucho-tokyo,3323217250,TCP,,32628,2323,198.20.69.98,US,United States,Illinois,IL,60661,41.8825,-87.6441,\n', '3/3/13 21:59,groucho-oregon,3730416887,TCP,,6000,1433,222.89.164.247,CN,China,Henan Sheng,41,,34.6836,113.5325,\n', '3/3/13 22:07,groucho-singapore,3738622573,TCP,,6000,3306,222.214.218.109,CN,China,Sichuan Sheng,51

In [12]:
def chunk_to_df(data_chunk):
    
    start = data_chunk.split('\\n')
    
    clean_step1 = start[0].strip(' \n').replace('=','').replace(f'dataset','')

    clean_step2 = clean_step1.split('\n+[')
    
    header = clean_step2[0]
    row1 = clean_step2[1]
    
    clean_row = []
    for dat in start[1:]:
        clean_row.append(dat.strip(" ',"))
        
    clean_row.insert(0,row1.strip("'"))
    clean_row.insert(0,header)
    
    clean_data_list = [data.split(',') for data in clean_row]
    
    chunk_df = pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])
    
    return chunk_df

In [13]:
chunk_to_df(prompt_made)

,datetime,host,src,proto,type,spt,dpt,srcstr,cc,country,locale,localeabbr,postalcode,latitude,longitude,
0,3/3/13 21:53,groucho-oregon,1032051418,TCP,,6000,1433,61.131.218.218,CN,China,Jiangxi Sheng,36,,28.55,115.9333,
1,3/3/13 21:57,groucho-oregon,1347834426,UDP,,5270,5060,80.86.82.58,DE,Germany,,,,51,9,None
2,3/3/13 21:58,groucho-oregon,2947856490,TCP,,2489,1080,175.180.184.106,TW,Taiwan,Taipei,,,25.0392,121.525,None
3,3/3/13 21:58,groucho-us-east,841842716,UDP,,43235,1900,50.45.128.28,US,United States,Oregon,OR,97124,45.5848,-122.9117,None
4,3/3/13 21:58,groucho-singapore,3587648279,TCP,,56577,80,213.215.43.23,FR,France,,,,48.86,2.35,None
5,3/3/13 21:58,groucho-tokyo,3323217250,TCP,,32628,2323,198.20.69.98,US,United States,Illinois,IL,60661,41.8825,-87.6441,None
6,3/3/13 21:59,groucho-oregon,3730416887,TCP,,6000,1433,222.89.164.247,CN,China,Henan Sheng,41,,34.6836,113.5325,None
7,3/3/13 22:07,groucho-singapore,3738622573,TCP,,6000,3306,222.214.218.109,CN,China,Sichuan Sheng,51,,30.6667,104.0667,None
8,3/3/13 22:12,groucho-oregon,3683919430,TCP,,6000,1433,219.148.38.70,CN,China,Hebei,13,,39.8897,115.275,None
9,3/3/13 22:14,groucho-singapore,1007884304,TCP,,6000,1433,60.19.24.16,CN,China,Liaoning,21,,41.7922,123.4328,None


In [23]:
def cgpt_reply(org,key,
                    role_player,
                    request_content,
                   file_name='database_tutor.txt'):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature = 0,
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}."}
            ]
        )
    with open(file_name,'a+') as sumry:
        sumry.write(f'Completion for \n {request_content}')
        sumry.write('******Completion starts****** \n')
        sumry.write(str(response['choices'][0]['message']['content']))
        sumry.write('*****Completion ends****** \n')
    return response['choices'][0]['message']['content']

In [24]:
cgpt_resp1 = cgpt_reply(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [25]:
print(cgpt_resp1)

1. To create a database table for this dataset in Postgres Database, we can use the following SQL command:

CREATE TABLE dataset (
    datetime TIMESTAMP,
    host VARCHAR(50),
    src BIGINT,
    proto VARCHAR(10),
    type VARCHAR(10),
    spt INT,
    dpt INT,
    srcstr VARCHAR(50),
    cc VARCHAR(10),
    country VARCHAR(50),
    locale VARCHAR(50),
    localeabbr VARCHAR(10),
    postalcode VARCHAR(10),
    latitude FLOAT,
    longitude FLOAT
);

2. To load the data from the CSV file into the newly created table, we can use the following command:

COPY dataset FROM '/path/to/dataset.csv' DELIMITER ',' CSV HEADER;

This assumes that the CSV file is located at '/path/to/dataset.csv' and that the columns are separated by commas and have a header row.

3. To normalize the table to 3rd normal form, we can split the table into three separate tables:

Table 1: hosts
- host_id (primary key)
- host_name

Table 2: locations
- location_id (primary key)
- country
- locale
- localeabbr
- post

### Getting DBA Administered

In [26]:
prompt_for_dba = """I have to administer the postgres database inside
AWS EC2 instance. Provide me the step by step process I need to take 
for installing, configuring, and the connecting to the Postgres instance."""

In [27]:
cgpt_resp2 = cgpt_reply(org=openai_org,key=openai_key,
                       role_player=get_role,
                        request_content=prompt_for_dba)

In [30]:
print(cgpt_resp2)

Sure, here are the step-by-step instructions for installing, configuring, and connecting to a Postgres instance on an AWS EC2 instance:

1. Launch an EC2 instance: First, launch an EC2 instance on AWS and select the appropriate operating system. For this example, we will use Amazon Linux 2.

2. Install Postgres: Once the instance is launched, connect to it using SSH and run the following command to install Postgres:

   ```
   sudo amazon-linux-extras install postgresql11
   ```

3. Configure Postgres: After installing Postgres, you need to configure it. Run the following command to initialize the database cluster:

   ```
   sudo postgresql-setup initdb
   ```

   This will create the necessary directories and files for the Postgres database.

4. Start Postgres: Once the database cluster is initialized, start the Postgres service using the following command:

   ```
   sudo systemctl start postgresql
   ```

5. Enable Postgres: To ensure that Postgres starts automatically on system bo

In [31]:
prompt_corrected = """Small correction. I have to install and connect to 
postgres database in ubuntu linux instance in EC2.Please provide 
the steps for doing"""

In [32]:
cgpt_resp3 = cgpt_reply(org=openai_org,key=openai_key,
                       role_player=get_role,
                        request_content=prompt_corrected)

In [33]:
print(cgpt_resp3)

Sure, here are the steps to install and connect to a PostgreSQL database in an Ubuntu Linux instance on EC2:

1. Launch an EC2 instance with Ubuntu Linux as the operating system.
2. Connect to the instance using SSH.
3. Update the package list by running the following command:

   ```
   sudo apt-get update
   ```

4. Install PostgreSQL by running the following command:

   ```
   sudo apt-get install postgresql postgresql-contrib
   ```

5. Once the installation is complete, start the PostgreSQL service by running the following command:

   ```
   sudo service postgresql start
   ```

6. By default, PostgreSQL creates a user called "postgres" with administrative privileges. Switch to the "postgres" user by running the following command:

   ```
   sudo su - postgres
   ```

7. Create a new database user by running the following command:

   ```
   createuser --interactive
   ```

   Follow the prompts to create a new user with the desired name and privileges.

8. Create a new database

### Lets check memory

In [40]:
prompt_star = """Thanks I will try that out. Refer to the 
tables hosts, locations and connections and create a star schema and
provide the sql query"""

In [36]:
cgpt_resp4 = cgpt_reply(org=openai_org,key=openai_key,
                       role_player=get_role,
                        request_content=prompt_star)

In [37]:
print(cgpt_resp4)

Sure, I can help you with that. 

First, let's define the star schema for the given tables:

- Fact table: connections
- Dimension tables: hosts, locations

The connections table will be the fact table as it contains the measures (connection_id, connection_time) and foreign keys to the dimension tables (host_id, location_id). The hosts and locations tables will be the dimension tables as they contain the attributes related to the hosts and locations.

Here is the SQL query to create the star schema:

```
CREATE TABLE hosts_dim (
    host_id INT PRIMARY KEY,
    host_name VARCHAR(50),
    host_type VARCHAR(50),
    host_location_id INT,
    FOREIGN KEY (host_location_id) REFERENCES locations_dim(location_id)
);

CREATE TABLE locations_dim (
    location_id INT PRIMARY KEY,
    location_name VARCHAR(50),
    location_city VARCHAR(50),
    location_state VARCHAR(50)
);

CREATE TABLE connections_fact (
    connection_id INT PRIMARY KEY,
    connection_time DATETIME,
    host_id INT,
    lo

In [39]:
prompt_queries = """Thanks I will try that schema. Kindly provide me 
3 SQL DML queries that uses the hosts_dim, locations_dim and 
connection_fact tables"""

In [43]:
cgpt_reply5 = cgpt_reply(org=openai_org,key=openai_key,
                       role_player=get_role,
                        request_content=prompt_queries)

In [44]:
print(cgpt_reply5)

Sure, here are three SQL DML queries that use the hosts_dim, locations_dim, and connection_fact tables:

1. Query to retrieve the total number of connections made by each host in a specific location:

```
SELECT h.hostname, l.location, COUNT(*) AS total_connections
FROM hosts_dim h
JOIN connection_fact c ON h.host_id = c.host_id
JOIN locations_dim l ON c.location_id = l.location_id
WHERE l.location = 'New York'
GROUP BY h.hostname, l.location;
```

2. Query to retrieve the average connection duration for each host in a specific location:

```
SELECT h.hostname, l.location, AVG(c.duration) AS avg_duration
FROM hosts_dim h
JOIN connection_fact c ON h.host_id = c.host_id
JOIN locations_dim l ON c.location_id = l.location_id
WHERE l.location = 'San Francisco'
GROUP BY h.hostname, l.location;
```

3. Query to retrieve the top 5 hosts with the highest number of connections:

```
SELECT h.hostname, COUNT(*) AS total_connections
FROM hosts_dim h
JOIN connection_fact c ON h.host_id = c.host_id
